In [1]:
# send a url request for chicago transit authority data via Bus Time API

# sample request url:
#http://www.ctabustracker.com/bustime/api/v2/gettime?key=89dj2he89d8j3j3ksjhdue93j

import json
import requests


config_path = "config.json"
with open(config_path,'r') as configfile:
    config = json.load(configfile)
api_key = config['auth']['api-key']

# 10566 - 55th & HP Blvd Westbound
# 1654 - 55 & HP Blvd  Southbound
# 1518 - 55 & HP Blvd Northbound

stop_ids = [10566,1654,1518]
route_ids = [6,55,171]


# construct a query for the API prediction call

In [2]:

# prefix for predictions request
prediction_url = 'http://www.ctabustracker.com/bustime/api/v2/getpredictions'

# add apikey
prediction_url += '?key=' + api_key



# add arguments containing stop id(s)
prediction_url += '&stpid='
for idx,stpid in enumerate(stop_ids):
    if idx > 0:
         prediction_url += ','
    prediction_url += str(stpid)

# and route arguments
prediction_url += '&rt='
for idx,rtid in enumerate(route_ids):
    if idx > 0:
         prediction_url += ','
    prediction_url += str(rtid)
    
prediction_url += '&format=json'
    
#params = dict(          #    <------------- todo this looks like a better way to format params
#    origin='Chicago,IL'
#)
    
# final html request
print(prediction_url)



http://www.ctabustracker.com/bustime/api/v2/getpredictions?key=A2HB9NmR4UCYyfB3HD6NfRCxB&stpid=10566,1654,1518&rt=6,55,171&format=json


# make a request, convert data to json, print out key data

In [3]:
# option 2 - with requests library through urllib3    
    

response = requests.get(url=prediction_url) # , params=params)
try:
    data = response.json() # Check the JSON Response Content documentation below
except:
    print("Warning, JSON Decode Error")
    print(response)
    
'''    
response_list = data['bustime-response']['prd']
for idx, response in enumerate(response_list):
    route,direction = response['rt'],response['rtdir']
    countdown, arrival_time = response['prdctdn'], response['prdtm']
    
    print("Route {} {}".format( response['rt'], response['rtdir']) )
    
    if countdown == 'DUE':
        print("DUE".format(response['prdctdn']) )
    else:
        print("{} minutes".format(response['prdctdn']) )
    
    print("Predicted to arrive at {}".format(response['prdtm']) )
    print()
'''

# a few useful fields-
# rt (route) rtdir (route direction)
# dly # true or false - if true prediction data may be inaccurate? todo look into this
# 'prdtm' # predicted time in minutes e.g. 20190430 18:43
# 'prdctdn' # minues until arrival or DUE e.g. 6
# 'des' # destination

Route 55 Westbound
DUE
Predicted to arrive at 20190302 19:28

Route 6 Northbound
6 minutes
Predicted to arrive at 20190302 19:33

Route 55 Eastbound
6 minutes
Predicted to arrive at 20190302 19:34

Route 6 Southbound
12 minutes
Predicted to arrive at 20190302 19:39

Route 6 Southbound
13 minutes
Predicted to arrive at 20190302 19:40

Route 55 Westbound
14 minutes
Predicted to arrive at 20190302 19:41

Route 6 Northbound
19 minutes
Predicted to arrive at 20190302 19:46

Route 55 Eastbound
23 minutes
Predicted to arrive at 20190302 19:50

Route 6 Southbound
23 minutes
Predicted to arrive at 20190302 19:50

Route 55 Westbound
28 minutes
Predicted to arrive at 20190302 19:55



## Collate all the routes into Separate 3-Line Messages for Printing

In [15]:
import pandas as pd

data = pd.DataFrame(data['bustime-response']['prd'])

In [49]:
routes_running = data['rt'].unique()

for iRoutes in routes_running:
    #initialize and clear LCD message
    text = "Route {}\n".format(iRoutes)
    
    # get data subset
    rt_data = data.loc[data['rt']==iRoutes]
    
    #get unique directions
    directions = rt_data['rtdir'].unique()
    
    for iDirection in directions:
        dir_data = rt_data.loc[rt_data['rtdir']==iDirection]
        
        # set time string
        times = ""
        
        for iTime in dir_data['prdctdn']:
            times += " {},".format(iTime)

        text += "{}: ".format(iDirection[:-5]) + times + '\n'
    

Route 55
West:  DUE, 14, 28,
East:  6, 23,

Route 6
North:  6, 19,
South:  12, 13, 23,

